# Test

In [1]:
import ollama
import json
import datetime
import logging
import pandas as pd

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configuration
MODEL_NAME = "llama3.2"

# --- Agent Class and Subclasses ---
class Agent:
    """
    Base class for all agents.
    """
    def __init__(self, role, model=MODEL_NAME):
        self.role = role
        self.model = model
        self.log = []
        self.tools = {}

    def generate_response(self, prompt, stream=False):
        """
        Generates a response using the Ollama LLM.
        """
        try:
            response = ollama.chat(model=self.model, messages=[{'role': 'user', 'content': prompt}], stream=stream)
            final_response = ""
            if stream:
                for chunk in response:
                    if 'content' in chunk['message']:
                        final_response += chunk['message']['content']
                        print(chunk['message']['content'], end='', flush=True)
            else:
                final_response = response['message']['content'].strip()
            self.log_action("Generated response", prompt)
            return final_response
        except Exception as e:
            logging.error(f"{self.role} - Error generating response: {e}")
            return None

    def log_action(self, action, details=None):
        """
        Logs the agent's action with a timestamp and level.
        """
        timestamp = datetime.datetime.now().isoformat()
        log_entry = {"timestamp": timestamp, "action": action, "details": details}
        self.log.append(log_entry)
        logging.info(f"{self.role}: {action} - {details if details else ''}")

    def add_tool(self, tool_name, tool_function):
        """
        Adds a tool (function) that the agent can use. For ReAct.
        """
        self.tools[tool_name] = tool_function

    def use_tool(self, tool_name, tool_input):
        """
        Executes a tool and logs the usage. For ReAct.
        """
        if tool_name in self.tools:
            result = self.tools[tool_name](tool_input)
            self.log_action(f"Used tool: {tool_name}", f"Input: {tool_input}, Result: {result}")
            return result
        else:
            self.log_action("Tool not found", tool_name)
            return None

class FundamentalsAnalyst(Agent):
    def __init__(self):
        super().__init__("Fundamentals Analyst")

    def analyze_fundamentals(self, company_data):
        """
        Analyzes company financials and returns a report.
        company_data should be a dict or DataFrame
        """
        prompt = f"Analyze the following company data and provide a fundamentals analysis report:\n{company_data}"
        report = self.generate_response(prompt)
        return report

class SentimentAnalyst(Agent):
    def __init__(self):
        super().__init__("Sentiment Analyst")

    def analyze_sentiment(self, social_media_data):
        """
        Analyzes social media data and returns a sentiment analysis.
        social_media_data should be a string
        """
        prompt = f"Analyze the following social media data and provide a sentiment analysis:\n{social_media_data}"
        sentiment = self.generate_response(prompt)
        return sentiment

class NewsAnalyst(Agent):
    def __init__(self):
        super().__init__("News Analyst")

    def analyze_news(self, news_data):
        """
        Analyzes news events and returns a report.
        news_data should be a string
        """
        prompt = f"Analyze the following news data and provide a report on potential market movements:\n{news_data}"
        report = self.generate_response(prompt)
        return report

class TechnicalAnalyst(Agent):
    def __init__(self):
        super().__init__("Technical Analyst")

    def analyze_technical_indicators(self, price_data):
        """
        Analyzes price data and returns a technical analysis.
        price_data should be a DataFrame with price and volume
        """
        prompt = f"Analyze the following price data and provide a technical analysis:\n{price_data}"
        analysis = self.generate_response(prompt)
        return analysis

class BullishResearcher(Agent):
    def __init__(self):
        super().__init__("Bullish Researcher")

    def analyze_for_bullish_signals(self, analysis_reports):
        """
        Analyzes reports for bullish signals and returns an assessment.
        analysis_reports should be a string
        """
        prompt = f"Analyze the following reports and provide an assessment focused on bullish signals:\n{analysis_reports}"
        assessment = self.generate_response(prompt)
        return assessment

class BearishResearcher(Agent):
    def __init__(self):
        super().__init__("Bearish Researcher")

    def analyze_for_bearish_signals(self, analysis_reports):
        """
        Analyzes reports for bearish signals and returns an assessment.
        analysis_reports should be a string
        """
        prompt = f"Analyze the following reports and provide an assessment focused on bearish signals:\n{analysis_reports}"
        assessment = self.generate_response(prompt)
        return assessment

class TraderAgent(Agent):
    def __init__(self, risk_profile="medium"):
        super().__init__("Trader Agent")
        self.risk_profile = risk_profile

    def make_trading_decision(self, bullish_assessment, bearish_assessment):
        """
        Makes a trading decision based on bullish and bearish assessments.
        Assessments should be strings.
        """
        prompt = f"Based on the bullish assessment: {bullish_assessment} and the bearish assessment: {bearish_assessment}, make a trading decision with a {self.risk_profile} risk profile."
        decision = self.generate_response(prompt)
        return decision

class RiskManagementTeam(Agent):
    def __init__(self):
        super().__init__("Risk Management Team")

    def assess_risk(self, trading_decision):
        """
        Assesses the risk of a trading decision.
        trading_decision should be a string.
        """
        prompt = f"Assess the risk associated with the following trading decision: {trading_decision}"
        risk_assessment = self.generate_response(prompt)
        return risk_assessment

class FundManager(Agent):
    def __init__(self):
        super().__init__("Fund Manager")

    def approve_trade(self, trading_decision, risk_assessment):
        """
        Approves or rejects a trade based on the decision and risk assessment.
        trading_decision and risk_assessment should be strings.
        """
        prompt = f"The trading decision is: {trading_decision} and the risk assessment is: {risk_assessment}. Approve or reject this trade."
        approval = self.generate_response(prompt)
        return approval

# --- Helper Functions ---
def load_data(file_path, file_type="csv"):
    """
    Loads data from a file (CSV or JSON).  Expand as needed.
    """
    try:
        if file_type == "csv":
            return pd.read_csv(file_path)
        elif file_type == "json":
            with open(file_path, 'r') as f:
                return json.load(f)
        else:
            logging.error(f"Unsupported file type: {file_type}")
            return None
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        return None
    except Exception as e:
        logging.error(f"Error loading data: {e}")
        return None

def save_logs(agents, log_dir="logs"):
    """
    Saves the logs for all agents to JSON files.
    """
    import os
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    for agent in agents:
        file_path = os.path.join(log_dir, f"{agent.role.replace(' ', '_')}_log.json")
        with open(file_path, 'w') as f:
            json.dump(agent.log, f, indent=4)
        logging.info(f"Saved log for {agent.role} to {file_path}")

# --- Example Usage ---
if __name__ == "__main__":
    # Initialize agents
    agents = {
        "fundamentals_analyst": FundamentalsAnalyst(),
        "sentiment_analyst": SentimentAnalyst(),
        "news_analyst": NewsAnalyst(),
        "technical_analyst": TechnicalAnalyst(),
        "bullish_researcher": BullishResearcher(),
        "bearish_researcher": BearishResearcher(),
        "trader_agent": TraderAgent(),
        "risk_management_team": RiskManagementTeam(),
        "fund_manager": FundManager()
    }

    # Simulate data loading
    company_data = {
        "AAPL": {"Q1_revenue": 100, "Q2_revenue": 105, "debt": 20}
    }
    social_media_data = "Trending on Twitter: #AAPLBuy strong positive sentiment"
    news_data = "Breaking: AAPL announces new product launch"
    price_data = pd.DataFrame({
        "timestamp": pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05']),
        "price": [150, 152, 155, 153, 156],
        "volume": [1000000, 1050000, 1100000, 950000, 1200000]
    })

    # Agent workflow
    fundamentals_report = agents["fundamentals_analyst"].analyze_fundamentals(company_data)
    sentiment_analysis = agents["sentiment_analyst"].analyze_sentiment(social_media_data)
    news_report = agents["news_analyst"].analyze_news(news_data)
    technical_analysis = agents["technical_analyst"].analyze_technical_indicators(price_data)

    analysis_reports = f"Fundamentals: {fundamentals_report}, Sentiment: {sentiment_analysis}, News: {news_report}, Technical: {technical_analysis}"

    bullish_assessment = agents["bullish_researcher"].analyze_for_bullish_signals(analysis_reports)
    bearish_assessment = agents["bearish_researcher"].analyze_for_bearish_signals(analysis_reports)

    trading_decision = agents["trader_agent"].make_trading_decision(bullish_assessment, bearish_assessment)
    risk_assessment = agents["risk_management_team"].assess_risk(trading_decision)
    approval = agents["fund_manager"].approve_trade(trading_decision, risk_assessment)

    print("Trading Process Completed.")

    # Save logs
    save_logs(agents.values())

2025-03-20 14:01:58,055 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-20 14:01:58,058 - INFO - Fundamentals Analyst: Generated response - Analyze the following company data and provide a fundamentals analysis report:
{'AAPL': {'Q1_revenue': 100, 'Q2_revenue': 105, 'debt': 20}}
2025-03-20 14:02:01,406 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-20 14:02:01,409 - INFO - Sentiment Analyst: Generated response - Analyze the following social media data and provide a sentiment analysis:
Trending on Twitter: #AAPLBuy strong positive sentiment
2025-03-20 14:02:08,327 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-20 14:02:08,330 - INFO - News Analyst: Generated response - Analyze the following news data and provide a report on potential market movements:
Breaking: AAPL announces new product launch
2025-03-20 14:02:15,196 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/

Trading Process Completed.


In [22]:
import ollama
import json
import datetime
import logging
import pandas as pd
import os
from langchain.output_parsers import StructuredOutputParser, ResponseSchema  # Import necessary classes

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configuration
MODEL_NAME = "llama3.2"
ALPHA_FACTOR_DIR = "outputs"  # Directory to search for CSV files

# --- Helper Functions ---
def load_data(file_path, file_type="csv"):
    """
    Loads data from a file (CSV or JSON). Expand as needed.
    """
    try:
        if file_type == "csv":
            return pd.read_csv(file_path)
        elif file_type == "json":
            with open(file_path, 'r') as f:
                return json.load(f)
        else:
            logging.error(f"Unsupported file type: {file_type}")
            return None
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        return None
    except Exception as e:
        logging.error(f"Error loading data: {e}")
        return None

def save_logs(agents, log_dir="logs"):
    """
    Saves the logs for all agents to JSON files.
    """
    import os
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    for agent in agents:
        file_path = os.path.join(log_dir, f"{agent.role.replace(' ', '_')}_log.json")
        with open(file_path, 'w') as f:
            json.dump(agent.log, f, indent=4)
        logging.info(f"Saved log for {agent.role} to {file_path}")

def load_alpha_factors(directory):
    """
    Searches for and loads all CSV files in a directory.
    Returns a dictionary with alpha factor names as keys and their codes as values.
    """
    alpha_factors = {}
    try:
        for filename in os.listdir(directory):
            if filename.endswith(".csv"):
                file_path = os.path.join(directory, filename)
                df = pd.read_csv(file_path)  # Directly use pandas to read CSV

                # Check if 'name' and 'code' columns exist
                if 'name' in df.columns and 'code' in df.columns:
                    # Iterate through the rows and create the dictionary
                    for index, row in df.iterrows():
                        alpha_name = row['name']
                        alpha_code = row['code']
                        alpha_factors[alpha_name] = alpha_code
                    logging.info(f"Loaded alpha factors from '{file_path}'")
                else:
                    logging.error(
                        f"Alpha factor file '{file_path}' missing required columns: name or code")
        if not alpha_factors:
            logging.warning(f"No valid alpha factor files found in '{directory}'")
    except FileNotFoundError:
        logging.error(f"Directory not found: {directory}")
    except Exception as e:
        logging.error(f"Error loading alpha factors: {e}")
    return alpha_factors

# --- Agent Class and Subclasses ---
class Agent:
    """
    Base class for all agents.
    """
    def __init__(self, role, model=MODEL_NAME):
        self.role = role
        self.model = model
        self.log = []
        self.tools = {}

    def generate_response(self, prompt, stream=False):
        """
        Generates a response using the Ollama LLM.
        """
        try:
            response = ollama.chat(model=self.model, messages=[{'role': 'user', 'content': prompt}], stream=stream)
            final_response = ""
            if stream:
                for chunk in response:
                    if 'content' in chunk['message']:
                        final_response += chunk['message']['content']
                        print(chunk['message']['content'], end='', flush=True)
            else:
                final_response = response['message']['content'].strip()
            self.log_action("Generated response", prompt)
            return final_response
        except Exception as e:
            logging.error(f"{self.role} - Error generating response: {e}")
            return None

    def log_action(self, action, details=None):
        """
        Logs the agent's action with a timestamp and level.
        """
        timestamp = datetime.datetime.now().isoformat()
        log_entry = {"timestamp": timestamp, "action": action, "details": details}
        self.log.append(log_entry)
        logging.info(f"{self.role}: {action} - {details if details else ''}")

    def add_tool(self, tool_name, tool_function):
        """
        Adds a tool (function) that the agent can use. For ReAct.
        """
        self.tools[tool_name] = tool_function

    def use_tool(self, tool_name, tool_input):
        """
        Executes a tool and logs the usage. For ReAct.
        """
        if tool_name in self.tools:
            result = self.tools[tool_name](tool_input)
            self.log_action(f"Used tool: {tool_name}", f"Input: {tool_input}, Result: {result}")
            return result
        else:
            self.log_action("Tool not found", tool_name)
            return None

class AlphaFactorAnalyst(Agent):  # New Agent
    def __init__(self):
        super().__init__("Alpha Factor Analyst")

    def analyze_alpha_factors(self, alpha_factors):
        """
        Analyzes a dictionary of alpha factors (DataFrames) and
        returns a dictionary in the specified format.
        """

        # Define the schema for the output
        response_schemas = [
            ResponseSchema(name="selected_alphas",
                           description="A list of dictionaries, where each dictionary contains the alpha_name and justification for selected alpha factors"),
            ResponseSchema(name="rejected_alphas",
                           description="A list of dictionaries, where each dictionary contains the alpha_name and justification for rejected alpha factors"),
        ]

        # Create an output parser
        output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

        prompt = f"""
        Analyze the following alpha factors and decide which ones outperform and which ones to reject. 
        Provide valid reasons for your decisions. The alpha factors are provided as a dictionary 
        where keys are alpha names and values are the alpha factor formulas:
        {alpha_factors}

        You MUST *put the outperform alpha factors into "selected_alphas"* and *put the rejected alpha factors into "rejected_alphas"*.

        You MUST *format your output as a dictionary with "selected_alphas" and "rejected_alphas"*. Each dictionary MUST have the key of "alpha_name" and the value of "justification".
        
        {output_parser.get_format_instructions()}
        """
        
        analysis_result = self.generate_response(prompt)
        try:
          return output_parser.parse(analysis_result)
        except Exception as e:
          logging.error(f"{self.role} - Error parsing alpha analysis: {e}")
          return {"selected_alphas": "", "rejected_alphas": ""} # Return empty lists in case of error

class FundamentalsAnalyst(Agent):
    def __init__(self):
        super().__init__("Fundamentals Analyst")

    def analyze_fundamentals(self, company_data):
        """
        Analyzes company financials and returns a report.
        company_data should be a dict or DataFrame
        """
        prompt = f"Analyze the following company data and provide a fundamentals analysis report:\n{company_data}"
        report = self.generate_response(prompt)
        return report

class SentimentAnalyst(Agent):
    def __init__(self):
        super().__init__("Sentiment Analyst")

    def analyze_sentiment(self, social_media_data):
        """
        Analyzes social media data and returns a sentiment analysis.
        social_media_data should be a string
        """
        prompt = f"Analyze the following social media data and provide a sentiment analysis:\n{social_media_data}"
        sentiment = self.generate_response(prompt)
        return sentiment

class NewsAnalyst(Agent):
    def __init__(self):
        super().__init__("News Analyst")

    def analyze_news(self, news_data):
        """
        Analyzes news events and returns a report.
        news_data should be a string
        """
        prompt = f"Analyze the following news data and provide a report on potential market movements:\n{news_data}"
        report = self.generate_response(prompt)
        return report

class TechnicalAnalyst(Agent):
    def __init__(self):
        super().__init__("Technical Analyst")

    def analyze_technical_indicators(self, price_data):
        """
        Analyzes price data and returns a technical analysis.
        price_data should be a DataFrame with price and volume
        """
        prompt = f"Analyze the following price data and provide a technical analysis:\n{price_data}"
        analysis = self.generate_response(prompt)
        return analysis

class BullishResearcher(Agent):
    def __init__(self):
        super().__init__("Bullish Researcher")

    def analyze_for_bullish_signals(self, analysis_reports):
        """
        Analyzes reports for bullish signals and returns an assessment.
        analysis_reports should be a string
        """
        prompt = f"Analyze the following reports and provide an assessment focused on bullish signals:\n{analysis_reports}"
        assessment = self.generate_response(prompt)
        return assessment

class BearishResearcher(Agent):
    def __init__(self):
        super().__init__("Bearish Researcher")

    def analyze_for_bearish_signals(self, analysis_reports):
        """
        Analyzes reports for bearish signals and returns an assessment.
        analysis_reports should be a string
        """
        prompt = f"Analyze the following reports and provide an assessment focused on bearish signals:\n{analysis_reports}"
        assessment = self.generate_response(prompt)
        return assessment

class TraderAgent(Agent):
    def __init__(self, risk_profile="medium"):
        super().__init__("Trader Agent")
        self.risk_profile = risk_profile

    def make_trading_decision(self, bullish_assessment, bearish_assessment, alpha_analysis):
        """
        Makes a trading decision based on bullish and bearish assessments,
        and now also alpha factor analysis.
        Assessments should be strings, alpha_analysis should be the dict
        from AlphaFactorAnalyst.
        """
        prompt = f"""Based on the bullish assessment: {bullish_assessment}, 
        the bearish assessment: {bearish_assessment}, and the alpha factor analysis: 
        {alpha_analysis}, make a trading decision with a {self.risk_profile} risk profile."""
        decision = self.generate_response(prompt)
        return decision

class RiskManagementTeam(Agent):
    def __init__(self):
        super().__init__("Risk Management Team")

    def assess_risk(self, trading_decision):
        """
        Assesses the risk of a trading decision.
        trading_decision should be a string.
        """
        prompt = f"Assess the risk associated with the following trading decision: {trading_decision}"
        risk_assessment = self.generate_response(prompt)
        return risk_assessment

class FundManager(Agent):
    def __init__(self):
        super().__init__("Fund Manager")

    def approve_trade(self, trading_decision, risk_assessment):
        """
        Approves or rejects a trade based on the decision and risk assessment.
        trading_decision and risk_assessment should be strings.
        """
        prompt = f"The trading decision is: {trading_decision} and the risk assessment is: {risk_assessment}. Approve or reject this trade."
        approval = self.generate_response(prompt)
        return approval

# --- Example Usage ---
"""
if __name__ == "__main__":
    # Initialize agents
    agents = {
        "alpha_factor_analyst": AlphaFactorAnalyst(), # Create Alpha Factor Agent
        "fundamentals_analyst": FundamentalsAnalyst(),
        "sentiment_analyst": SentimentAnalyst(),
        "news_analyst": NewsAnalyst(),
        "technical_analyst": TechnicalAnalyst(),
        "bullish_researcher": BullishResearcher(),
        "bearish_researcher": BearishResearcher(),
        "trader_agent": TraderAgent(),
        "risk_management_team": RiskManagementTeam(),
        "fund_manager": FundManager()
    }

    # Load alpha factors
    alpha_factors = load_alpha_factors(ALPHA_FACTOR_DIR)  # Load from directory
    if not alpha_factors:
        logging.warning("No alpha factors loaded. Exiting.")
        exit()

    # Simulate data loading (replace with your data sources)
    company_data = {
        "AAPL": {"Q1_revenue": 100, "Q2_revenue": 105, "debt": 20}
    }
    social_media_data = "Trending on Twitter: #AAPLBuy strong positive sentiment"
    news_data = "Breaking: AAPL announces new product launch"
    price_data = pd.DataFrame({
        "timestamp": pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05']),
        "price": [150, 152, 155, 153, 156],
        "volume": [1000000, 1050000, 1100000, 950000, 1200000]
    })

    # Agent workflow
    alpha_analysis = agents["alpha_factor_analyst"].analyze_alpha_factors(alpha_factors) # Analyze alpha factors first
    fundamentals_report = agents["fundamentals_analyst"].analyze_fundamentals(company_data)
    sentiment_analysis = agents["sentiment_analyst"].analyze_sentiment(social_media_data)
    news_report = agents["news_analyst"].analyze_news(news_data)
    technical_analysis = agents["technical_analyst"].analyze_technical_indicators(price_data)

    analysis_reports = f"Fundamentals: {fundamentals_report}, Sentiment: {sentiment_analysis}, News: {news_report}, Technical: {technical_analysis}"

    bullish_assessment = agents["bullish_researcher"].analyze_for_bullish_signals(analysis_reports)
    bearish_assessment = agents["bearish_researcher"].analyze_for_bearish_signals(analysis_reports)

    trading_decision = agents["trader_agent"].make_trading_decision(bullish_assessment, bearish_assessment, alpha_analysis) # Pass alpha analysis to trader
    risk_assessment = agents["risk_management_team"].assess_risk(trading_decision)
    approval = agents["fund_manager"].approve_trade(trading_decision, risk_assessment)

    print("Trading Process Completed.")

    # Save logs
    save_logs(agents.values())"
"""
if __name__ == "__main__":
    # Initialize agents
    agents = {
        "alpha_factor_analyst": AlphaFactorAnalyst(),
        "fundamentals_analyst": FundamentalsAnalyst(),
        "sentiment_analyst": SentimentAnalyst(),
        "news_analyst": NewsAnalyst(),
        "technical_analyst": TechnicalAnalyst(),
        "bullish_researcher": BullishResearcher(),
        "bearish_researcher": BearishResearcher(),
        "trader_agent": TraderAgent(),
        "risk_management_team": RiskManagementTeam(),
        "fund_manager": FundManager()
    }

    # Load alpha factors
    alpha_factors = load_alpha_factors(ALPHA_FACTOR_DIR)
    if not alpha_factors:
        logging.warning("No alpha factors loaded. Exiting.")
        exit()

    # Simulate data loading
    company_data = {
        "AAPL": {"Q1_revenue": 100, "Q2_revenue": 105, "debt": 20}
    }
    social_media_data = "Trending on Twitter: #AAPLBuy strong positive sentiment"
    news_data = "Breaking: AAPL announces new product launch"
    price_data = pd.DataFrame({
        "timestamp": pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05']),
        "price": [150, 152, 155, 153, 156],
        "volume": [1000000, 1050000, 1100000, 950000, 1200000]
    })

    # Agent workflow
    alpha_analysis = agents["alpha_factor_analyst"].analyze_alpha_factors(alpha_factors)

    # Accessing the results later
    selected_alphas = alpha_analysis.get("selected_alphas")
    rejected_alphas = alpha_analysis.get("rejected_alphas")

    # Use the results
    if selected_alphas:
        print("Selected Alpha Factors:")
        for alpha in selected_alphas:
            print(f"{alpha['alpha_name']}: {alpha['justification']}")

    if rejected_alphas:
        print("Rejected Alpha Factors:")
        for alpha in rejected_alphas:
            print(f"{alpha['alpha_name']}: {alpha['justification']}")

    fundamentals_report = agents["fundamentals_analyst"].analyze_fundamentals(company_data)
    sentiment_analysis = agents["sentiment_analyst"].analyze_sentiment(social_media_data)
    news_report = agents["news_analyst"].analyze_news(news_data)
    technical_analysis = agents["technical_analyst"].analyze_technical_indicators(price_data)

    analysis_reports = f"Fundamentals: {fundamentals_report}, Sentiment: {sentiment_analysis}, News: {news_report}, Technical: {technical_analysis}"

    bullish_assessment = agents["bullish_researcher"].analyze_for_bullish_signals(analysis_reports)
    bearish_assessment = agents["bearish_researcher"].analyze_for_bearish_signals(analysis_reports)

    trading_decision = agents["trader_agent"].make_trading_decision(bullish_assessment, bearish_assessment, alpha_analysis)
    risk_assessment = agents["risk_management_team"].assess_risk(trading_decision)
    approval = agents["fund_manager"].approve_trade(trading_decision, risk_assessment)

    print("Trading Process Completed.")

    # Save logs
    save_logs(agents.values())

2025-03-20 14:43:12,357 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_2.csv'
2025-03-20 14:43:12,359 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_3.csv'
2025-03-20 14:43:12,361 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_1.csv'
2025-03-20 14:43:12,362 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_4.csv'


2025-03-20 14:43:12,363 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_5.csv'
2025-03-20 14:43:30,444 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-20 14:43:30,444 - INFO - Alpha Factor Analyst: Generated response - 
        Analyze the following alpha factors and decide which ones outperform and which ones to reject. 
        Provide valid reasons for your decisions. The alpha factors are provided as a dictionary 
        where keys are alpha names and values are the alpha factor formulas:
        {'Price Momentum (14 days)': '((CLOSE - DELAY(CLOSE, 14)) / DELAY(CLOSE, 14))', 'Mean Reversion (20 days)': '(MEAN(CLOSE, 20) - CLOSE)', '20-Day Volatility': 'STD(CLOSE, 20)', 'Price-to-Earnings Ratio (P/E)': '(CLOSE / EPS)', 'Trading Volume': 'VOLUME', 'Gross Profit Margin': '(GROSS_PROFIT / REVENUE)', 'Earnings Growth Rate': '(EPS / DELAY(EPS,1) - 1)', 'Moving Average (MA)': 'SMA(CLOSE, 20)', 'GDP Growth Rate': 'GDP - DE

Trading Process Completed.


In [23]:
for alpha in rejected_alphas:
    print(f" {alpha['alpha_name']}: {alpha['justification']}")

In [24]:
for alpha in selected_alphas:
    print(f" {alpha['alpha_name']}: {alpha['justification']}")

# Template

In [19]:
def load_alpha_factors(directory):
    """
    Searches for and loads all CSV files in a directory.
    Returns a dictionary with alpha factor names as keys and their codes as values.
    """
    alpha_factors = {}
    try:
        for filename in os.listdir(directory):
            if filename.endswith(".csv"):
                file_path = os.path.join(directory, filename)
                df = pd.read_csv(file_path)  # Directly use pandas to read CSV

                # Check if 'name' and 'code' columns exist
                if 'name' in df.columns and 'code' in df.columns:
                    # Iterate through the rows and create the dictionary
                    for index, row in df.iterrows():
                        alpha_name = row['name']
                        alpha_code = row['code']
                        alpha_factors[alpha_name] = alpha_code
                    logging.info(f"Loaded alpha factors from '{file_path}'")
                else:
                    logging.error(
                        f"Alpha factor file '{file_path}' missing required columns: name or code")
        if not alpha_factors:
            logging.warning(f"No valid alpha factor files found in '{directory}'")
    except FileNotFoundError:
        logging.error(f"Directory not found: {directory}")
    except Exception as e:
        logging.error(f"Error loading alpha factors: {e}")
    return alpha_factors

# Load alpha factors
alpha_factors = load_alpha_factors(ALPHA_FACTOR_DIR)
if not alpha_factors:
    logging.warning("No alpha factors loaded. Exiting.")
    exit()

2025-03-20 14:38:10,854 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_2.csv'
2025-03-20 14:38:10,856 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_3.csv'
2025-03-20 14:38:10,857 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_1.csv'
2025-03-20 14:38:10,859 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_4.csv'
2025-03-20 14:38:10,860 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_5.csv'


In [21]:
len(alpha_factors)

28

# New try

In [29]:
import ollama
import json
import datetime
import logging
import pandas as pd
import os
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configuration
MODEL_NAME = "llama3.2"
ALPHA_FACTOR_DIR = "outputs"

# --- Helper Functions ---
def load_data(file_path, file_type="csv"):
    """
    Loads data from a file (CSV or JSON). Expand as needed.
    """
    try:
        if file_type == "csv":
            return pd.read_csv(file_path)
        elif file_type == "json":
            with open(file_path, 'r') as f:
                return json.load(f)
        else:
            logging.error(f"Unsupported file type: {file_type}")
            return None
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        return None
    except Exception as e:
        logging.error(f"Error loading data: {e}")
        return None

def save_logs(agents, log_dir="logs"):
    """
    Saves the logs for all agents to JSON files.
    """
    import os
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    for agent in agents:
        file_path = os.path.join(log_dir, f"{agent.role.replace(' ', '_')}_log.json")
        with open(file_path, 'w') as f:
            json.dump(agent.log, f, indent=4)
        logging.info(f"Saved log for {agent.role} to {file_path}")

def load_alpha_factors(directory):
    """
    Searches for and loads all CSV files in a directory.
    Returns a dictionary with alpha factor names as keys and their codes as values.
    """
    alpha_factors = {}
    try:
        for filename in os.listdir(directory):
            if filename.endswith(".csv"):
                file_path = os.path.join(directory, filename)
                df = pd.read_csv(file_path)

                if 'name' in df.columns and 'code' in df.columns:
                    for index, row in df.iterrows():
                        alpha_name = row['name']
                        alpha_code = row['code']
                        alpha_factors[alpha_name] = alpha_code
                    logging.info(f"Loaded alpha factors from '{file_path}'")
                else:
                    logging.error(
                        f"Alpha factor file '{file_path}' missing required columns: name or code")
        if not alpha_factors:
            logging.warning(f"No valid alpha factor files found in '{directory}'")
    except FileNotFoundError:
        logging.error(f"Directory not found: {directory}")
    except Exception as e:
        logging.error(f"Error loading alpha factors: {e}")
    return alpha_factors

# --- Agent Class and Subclasses ---
class Agent:
    """
    Base class for all agents.
    """
    def __init__(self, role, model=MODEL_NAME):
        self.role = role
        self.model = model
        self.log = []
        self.tools = {}

    def generate_response(self, prompt, stream=False):
        """
        Generates a response using the Ollama LLM.
        """
        try:
            response = ollama.chat(model=self.model, messages=[{'role': 'user', 'content': prompt}], stream=stream)
            final_response = ""
            if stream:
                for chunk in response:
                    if 'content' in chunk['message']:
                        final_response += chunk['message']['content']
                        print(chunk['message']['content'], end='', flush=True)
            else:
                final_response = response['message']['content'].strip()
            self.log_action("Generated response", prompt)
            return final_response
        except Exception as e:
            logging.error(f"{self.role} - Error generating response: {e}")
            return None

    def log_action(self, action, details=None):
        """
        Logs the agent's action with a timestamp and level.
        """
        timestamp = datetime.datetime.now().isoformat()
        log_entry = {"timestamp": timestamp, "action": action, "details": details}
        self.log.append(log_entry)
        logging.info(f"{self.role}: {action} - {details if details else ''}")

    def add_tool(self, tool_name, tool_function):
        """
        Adds a tool (function) that the agent can use. For ReAct.
        """
        self.tools[tool_name] = tool_function

    def use_tool(self, tool_name, tool_input):
        """
        Executes a tool and logs the usage. For ReAct.
        """
        if tool_name in self.tools:
            result = self.tools[tool_name](tool_input)
            self.log_action(f"Used tool: {tool_name}", f"Input: {tool_input}, Result: {result}")
            return result
        else:
            self.log_action("Tool not found", tool_name)
            return None

class MarketDebateAgent(Agent):
    """
    This agent orchestrates the debate between other agents.
    """

    def __init__(self):
        super().__init__("Market Debate Agent")
        self.analyst_agents = {  # Register analyst agents
            "fundamentals_analyst": FundamentalsAnalyst(),
            "sentiment_analyst": SentimentAnalyst(),
            "news_analyst": NewsAnalyst(),
            "technical_analyst": TechnicalAnalyst()
        }
        self.researcher_agents = { # Register researcher agents
            "bullish_researcher": BullishResearcher(),
            "bearish_researcher": BearishResearcher()
        }

    def conduct_debate(self, company_data, social_media_data, news_data, price_data, alpha_factors):
        """
        Orchestrates the market debate involving analyst and researcher agents.
        """
        debate_log = []

        # 1. Analyst Agents Provide Initial Analysis
        analyst_reports = {}
        for role, agent in self.analyst_agents.items():
            if role == "fundamentals_analyst":
                report = agent.analyze_fundamentals(company_data)
            elif role == "sentiment_analyst":
                report = agent.analyze_sentiment(social_media_data)
            elif role == "news_analyst":
                report = agent.analyze_news(news_data)
            elif role == "technical_analyst":
                report = agent.analyze_technical_indicators(price_data)
            else:
                continue  # Skip unknown roles
            analyst_reports[role] = report
            debate_log.append({"role": role, "report": report})
            self.log_action(f"{role} report", report)

        # 2. Researcher Agents Debate
        bullish_args = self.researcher_agents["bullish_researcher"].analyze_for_bullish_signals(
            json.dumps(analyst_reports))  # Pass reports as JSON string
        bearish_args = self.researcher_agents["bearish_researcher"].analyze_for_bearish_signals(
            json.dumps(analyst_reports))  # Pass reports as JSON string

        debate_log.append({"role": "bullish_researcher", "arguments": bullish_args})
        debate_log.append({"role": "bearish_researcher", "arguments": bearish_args})
        self.log_action("Bullish arguments", bullish_args)
        self.log_action("Bearish arguments", bearish_args)

        # 3. Alpha Factor Analysis (Integrated in Debate)
        alpha_analysis_prompt = f"""
        Given the analyst reports: {json.dumps(analyst_reports)}, 
        the bullish arguments: {bullish_args}, 
        the bearish arguments: {bearish_args}, 
        and the following alpha factors: {alpha_factors}, 
        which alpha factors should be selected or rejected and why?
        """
        alpha_analysis = self.generate_response(alpha_analysis_prompt)
        debate_log.append({"role": "debate_summary", "alpha_analysis": alpha_analysis})
        self.log_action("Alpha factor analysis", alpha_analysis)

        return debate_log

class FundamentalsAnalyst(Agent):
    def __init__(self):
        super().__init__("Fundamentals Analyst")

    def analyze_fundamentals(self, company_data):
        """
        Analyzes company financials and returns a report.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        company_data should be a dict or DataFrame
        """
        prompt = f"Analyze the following company data and provide a fundamentals analysis report:\n{company_data}"
        report = self.generate_response(prompt)
        return report

class SentimentAnalyst(Agent):
    def __init__(self):
        super().__init__("Sentiment Analyst")

    def analyze_sentiment(self, social_media_data):
        """
        Analyzes social media data and returns a sentiment analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        social_media_data should be a string
        """
        prompt = f"Analyze the following social media data and provide a sentiment analysis:\n{social_media_data}"
        sentiment = self.generate_response(prompt)
        return sentiment

class NewsAnalyst(Agent):
    def __init__(self):
        super().__init__("News Analyst")

    def analyze_news(self, news_data):
        """
        Analyzes news events and returns a report.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        news_data should be a string
        """
        prompt = f"Analyze the following news data and provide a report on potential market movements:\n{news_data}"
        report = self.generate_response(prompt)
        return report

class TechnicalAnalyst(Agent):
    def __init__(self):
        super().__init__("Technical Analyst")

    def analyze_technical_indicators(self, price_data):
        """
        Analyzes price data and returns a technical analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        price_data should be a DataFrame with price and volume
        """
        prompt = f"Analyze the following price data and provide a technical analysis:\n{price_data}"
        analysis = self.generate_response(prompt)
        return analysis

class BullishResearcher(Agent):
    def __init__(self):
        super().__init__("Bullish Researcher")

    def analyze_for_bullish_signals(self, analysis_reports):
        """
        Analyzes reports for bullish signals and returns an assessment.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        analysis_reports should be a string
        """
        prompt = f"Analyze the following reports and provide an assessment focused on bullish signals:\n{analysis_reports}"
        assessment = self.generate_response(prompt)
        return assessment

class BearishResearcher(Agent):
    def __init__(self):
        super().__init__("Bearish Researcher")

    def analyze_for_bearish_signals(self, analysis_reports):
        """
        Analyzes reports for bearish signals and returns an assessment.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        analysis_reports should be a string
        """
        prompt = f"Analyze the following reports and provide an assessment focused on bearish signals:\n{analysis_reports}"
        assessment = self.generate_response(prompt)
        return assessment

class TraderAgent(Agent):
    def __init__(self, risk_profile="medium"):
        super().__init__("Trader Agent")
        self.risk_profile = risk_profile

    def make_trading_decision(self, debate_log):
        """
        Makes a trading decision based on the debate log.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        debate_log should be a list of dictionaries.
        """
        prompt = f"""Based on the market debate: {json.dumps(debate_log)}, 
        make a trading decision with a {self.risk_profile} risk profile."""
        decision = self.generate_response(prompt)
        return decision

class RiskManagementTeam(Agent):
    def __init__(self):
        super().__init__("Risk Management Team")

    def assess_risk(self, trading_decision):
        """
        Assesses the risk of a trading decision.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        trading_decision should be a string.
        """
        prompt = f"Assess the risk associated with the following trading decision: {trading_decision}"
        risk_assessment = self.generate_response(prompt)
        return risk_assessment

class FundManager(Agent):
    def __init__(self):
        super().__init__("Fund Manager")

    def approve_trade(self, trading_decision, risk_assessment):
        """
        Approves or rejects a trade based on the decision and risk assessment.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        trading_decision and risk_assessment should be strings.
        """
        prompt = f"The trading decision is: {trading_decision} and the risk assessment is: {risk_assessment}. Approve or reject this trade."
        approval = self.generate_response(prompt)
        return approval

# --- Example Usage ---
if __name__ == "__main__":
    # Initialize agents
    agents = {
        "market_debate_agent": MarketDebateAgent(),  # Debate orchestrator
        "trader_agent": TraderAgent(),
        "risk_management_team": RiskManagementTeam(),
        "fund_manager": FundManager()
    }

    # Load alpha factors
    alpha_factors = load_alpha_factors(ALPHA_FACTOR_DIR)
    if not alpha_factors:
        logging.warning("No alpha factors loaded. Exiting.")
        exit()

    # Simulate data loading
    company_data = {
        "AAPL": {"Q1_revenue": 100, "Q2_revenue": 105, "debt": 20}
    }
    social_media_data = "Trending on Twitter: #AAPLBuy strong positive sentiment"
    news_data = "Breaking: AAPL announces new product launch"
    price_data = pd.DataFrame({
        "timestamp": pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05']),
        "price": [150, 152, 155, 153, 156],
        "volume": [1000000, 1050000, 1100000, 950000, 1200000]
    })

    # Agent workflow
    debate_log = agents["market_debate_agent"].conduct_debate(
        company_data, social_media_data, news_data, price_data, alpha_factors)  # Conduct the debate

    trading_decision = agents["trader_agent"].make_trading_decision(debate_log)  # Trader uses debate log
    risk_assessment = agents["risk_management_team"].assess_risk(trading_decision)
    approval = agents["fund_manager"].approve_trade(trading_decision, risk_assessment)

    print("Trading Process Completed.")

    # Save logs
    save_logs(agents.values())

2025-03-20 14:58:55,323 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_2.csv'
2025-03-20 14:58:55,324 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_3.csv'
2025-03-20 14:58:55,325 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_1.csv'
2025-03-20 14:58:55,327 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_4.csv'
2025-03-20 14:58:55,328 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_5.csv'
2025-03-20 14:59:01,121 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-20 14:59:01,123 - INFO - Fundamentals Analyst: Generated response - Analyze the following company data and provide a fundamentals analysis report:
{'AAPL': {'Q1_revenue': 100, 'Q2_revenue': 105, 'debt': 20}}
2025-03-20 14:59:01,123 - INFO - Market Debate Agent: fundamentals_analyst report - **Fundamentals Analysis Report**

**Company:** Apple Inc. 

Trading Process Completed.


In [28]:
# Review alpha factor results
for entry in debate_log:
    if entry["role"] == "debate_summary":
        alpha_analysis = entry["alpha_analysis"]
        print("Alpha Factor Analysis Results:")
        print(alpha_analysis)  # Print the raw output
        # Or, process and print in a more readable format
        # For example, if it's a JSON string:
        # alpha_analysis_dict = json.loads(alpha_analysis)
        # print(json.dumps(alpha_analysis_dict, indent=4))

Alpha Factor Analysis Results:
The choice of alpha factors depends on the specific investment strategy, risk tolerance, and market conditions. Here's a brief evaluation of each factor:

**Relevant Factors:**

1. **Price Momentum (14 days)**: This factor measures the rate of change in price over the past 14 days. It can be useful for identifying stocks with strong upward momentum.
2. **Mean Reversion (20 days)**: This factor checks if the stock's current price is deviating from its mean price over the past 20 days. It can help identify undervalued or overvalued stocks.
3. **20-Day Volatility**: Measuring volatility helps assess a stock's price movements and potential risk.
4. **Relative Strength Index (RSI)**: The RSI measures the magnitude of recent price changes to determine overbought or oversold conditions.
5. **Bollinger Bands (20, 2)**: This factor uses Bollinger Bands to measure volatility and identify potential breakouts.

**Factors with Limited Relevance:**

1. **GDP Growth Rat

# Final version

In [ ]:
import ollama
import json
import datetime
import logging
import pandas as pd
import os
import re  # Import the re module for post-processing
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configuration
MODEL_NAME = "llama3.2"
ALPHA_FACTOR_DIR = "outputs"

# --- Helper Functions ---
def load_data(file_path, file_type="csv"):
    """
    Loads data from a file (CSV or JSON). Expand as needed.
    """
    try:
        if file_type == "csv":
            return pd.read_csv(file_path)
        elif file_type == "json":
            with open(file_path, 'r') as f:
                return json.load(f)
        else:
            logging.error(f"Unsupported file type: {file_type}")
            return None
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        return None
    except Exception as e:
        logging.error(f"Error loading data: {e}")
        return None

def save_logs(agents, log_dir="logs"):
    """
    Saves the logs for all agents to JSON files.
    """
    import os
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    for agent in agents:
        file_path = os.path.join(log_dir, f"{agent.role.replace(' ', '_')}_log.json")
        with open(file_path, 'w') as f:
            json.dump(agent.log, f, indent=4)
        logging.info(f"Saved log for {agent.role} to {file_path}")

def load_alpha_factors(directory):
    """
    Searches for and loads all CSV files in a directory.
    Returns a dictionary with alpha factor names as keys and their codes as values.
    """
    alpha_factors = {}
    try:
        for filename in os.listdir(directory):
            if filename.endswith(".csv"):
                file_path = os.path.join(directory, filename)
                df = pd.read_csv(file_path)

                if 'name' in df.columns and 'code' in df.columns:
                    for index, row in df.iterrows():
                        alpha_name = row['name']
                        alpha_code = row['code']
                        alpha_factors[alpha_name] = alpha_code
                    logging.info(f"Loaded alpha factors from '{file_path}'")
                else:
                    logging.error(
                        f"Alpha factor file '{file_path}' missing required columns: name or code")
        if not alpha_factors:
            logging.warning(f"No valid alpha factor files found in '{directory}'")
    except FileNotFoundError:
        logging.error(f"Directory not found: {directory}")
    except Exception as e:
        logging.error(f"Error loading alpha factors: {e}")
    return alpha_factors

# --- Agent Class and Subclasses ---
class Agent:
    """
    Base class for all agents.
    """
    def __init__(self, role, model=MODEL_NAME):
        self.role = role
        self.model = model
        self.log = []
        self.tools = {}

    def generate_response(self, prompt, stream=False):
        """
        Generates a response using the Ollama LLM.
        """
        try:
            response = ollama.chat(model=self.model, messages=[{'role': 'user', 'content': prompt}], stream=stream)
            final_response = ""
            if stream:
                for chunk in response:
                    if 'content' in chunk['message']:
                        final_response += chunk['message']['content']
                        print(chunk['message']['content'], end='', flush=True)
            else:
                final_response = response['message']['content'].strip()
            self.log_action("Generated response", prompt)
            return final_response
        except Exception as e:
            logging.error(f"{self.role} - Error generating response: {e}")
            return None

    def log_action(self, action, details=None):
        """
        Logs the agent's action with a timestamp and level.
        """
        timestamp = datetime.datetime.now().isoformat()
        log_entry = {"timestamp": timestamp, "action": action, "details": details}
        self.log.append(log_entry)
        logging.info(f"{self.role}: {action} - {details if details else ''}")

    def add_tool(self, tool_name, tool_function):
        """
        Adds a tool (function) that the agent can use. For ReAct.
        """
        self.tools[tool_name] = tool_function

    def use_tool(self, tool_name, tool_input):
        """
        Executes a tool and logs the usage. For ReAct.
        """
        if tool_name in self.tools:
            result = self.tools[tool_name](tool_input)
            self.log_action(f"Used tool: {tool_name}", f"Input: {tool_input}, Result: {result}")
            return result
        else:
            self.log_action("Tool not found", tool_name)
            return None

class MarketDebateAgent(Agent):
    """
    This agent orchestrates the debate between other agents.
    """

    def __init__(self):
        super().__init__("Market Debate Agent")
        self.analyst_agents = {  # Register analyst agents
            "fundamentals_analyst": FundamentalsAnalyst(),
            "sentiment_analyst": SentimentAnalyst(),
            "news_analyst": NewsAnalyst(),
            "technical_analyst": TechnicalAnalyst()
        }
        self.researcher_agents = { # Register researcher agents
            "bullish_researcher": BullishResearcher(),
            "bearish_researcher": BearishResearcher()
        }

    def conduct_debate(self, company_data, social_media_data, news_data, price_data, alpha_factors):
        """
        Orchestrates the market debate involving analyst and researcher agents.
        """
        debate_log = []

        # 1. Analyst Agents Provide Initial Analysis
        analyst_reports = {}
        for role, agent in self.analyst_agents.items():
            if role == "fundamentals_analyst":
                report = agent.analyze_fundamentals(company_data)
            elif role == "sentiment_analyst":
                report = agent.analyze_sentiment(social_media_data)
            elif role == "news_analyst":
                report = agent.analyze_news(news_data)
            elif role == "technical_analyst":
                report = agent.analyze_technical_indicators(price_data)
            else:
                continue  # Skip unknown roles
            analyst_reports[role] = report
            debate_log.append({"role": role, "report": report})
            self.log_action(f"{role} report", report)

        # 2. Researcher Agents Debate
        formatted_reports = f"""
        Fundamentals Report: {analyst_reports['fundamentals_analyst']}
        Sentiment Report: {analyst_reports['sentiment_analyst']}
        News Report: {analyst_reports['news_analyst']}
        Technical Report: {analyst_reports['technical_analyst']}
        """
        bullish_args = self.researcher_agents["bullish_researcher"].analyze_for_bullish_signals(
            formatted_reports)
        bearish_args = self.researcher_agents["bearish_researcher"].analyze_for_bearish_signals(
            formatted_reports)

        debate_log.append({"role": "bullish_researcher", "arguments": bullish_args})
        debate_log.append({"role": "bearish_researcher", "arguments": bearish_args})
        self.log_action("Bullish arguments", bullish_args)
        self.log_action("Bearish arguments", bearish_args)

        # 3. Alpha Factor Analysis (Integrated in Debate)
        alpha_analysis_prompt = f"""
        Given the analyst reports: 
        Fundamentals Report: {analyst_reports['fundamentals_analyst']}, 
        Sentiment Report: {analyst_reports['sentiment_analyst']}, 
        News Report: {analyst_reports['news_analyst']}, 
        Technical Report: {analyst_reports['technical_analyst']},
        the bullish arguments: {bullish_args}, 
        the bearish arguments: {bearish_args}, 
        and the following alpha factors: {json.dumps(alpha_factors)}, 
        which alpha factors should be selected or rejected and why?
        """
        alpha_analysis = self.generate_response(alpha_analysis_prompt)
        debate_log.append({"role": "debate_summary", "alpha_analysis": alpha_analysis})
        self.log_action("Alpha factor analysis", alpha_analysis)

        return debate_log

class FundamentalsAnalyst(Agent):
    def __init__(self):
        super().__init__("Fundamentals Analyst")

    def analyze_fundamentals(self, company_data):
        """
        Analyzes company financials and returns a report.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        company_data should be a dict or DataFrame
        """
        prompt = f"""Analyze the following company data and provide a fundamentals analysis report.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Company Data: {company_data}"""
        report = self.generate_response(prompt)
        report = self.remove_code(report)  # Clean the output
        return report

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class SentimentAnalyst(Agent):
    def __init__(self):
        super().__init__("Sentiment Analyst")

    def analyze_sentiment(self, social_media_data):
        """
        Analyzes social media data and returns a sentiment analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        social_media_data should be a string
        """
        prompt = f"""Analyze the following social media data and provide a sentiment analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Social Media Data: {social_media_data}"""
        sentiment = self.generate_response(prompt)
        sentiment = self.remove_code(sentiment)  # Clean the output
        return sentiment

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class NewsAnalyst(Agent):
    def __init__(self):
        super().__init__("News Analyst")

    def analyze_news(self, news_data):
        """
        Analyzes news events and returns a report.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        news_data should be a string
        """
        prompt = f"""Analyze the following news data and provide a report on potential market movements.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        News Data: {news_data}"""
        report = self.generate_response(prompt)
        report = self.remove_code(report)  # Clean the output
        return report

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class TechnicalAnalyst(Agent):
    def __init__(self):
        super().__init__("Technical Analyst")

    def analyze_technical_indicators(self, price_data):
        """
        Analyzes price data and returns a technical analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        price_data should be a DataFrame with price and volume
        """
        prompt = f"""Analyze the following price data and provide a technical analysis.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Price Data: {price_data}"""
        analysis = self.generate_response(prompt)
        analysis = self.remove_code(analysis)  # Clean the output
        return analysis

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class BullishResearcher(Agent):
    def __init__(self):
        super().__init__("Bullish Researcher")

    def analyze_for_bullish_signals(self, analysis_reports):
        """
        Analyzes reports for bullish signals and returns an assessment.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        analysis_reports should be a string
        """
        prompt = f"""Analyze the following reports and provide an assessment focused on bullish signals.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Analysis Reports: {analysis_reports}"""
        assessment = self.generate_response(prompt)
        assessment = self.remove_code(assessment)  # Clean the output
        return assessment

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class BearishResearcher(Agent):
    def __init__(self):
        super().__init__("Bearish Researcher")

    def analyze_for_bearish_signals(self, analysis_reports):
        """
        Analyzes reports for bearish signals and returns an assessment focused on bearish signals.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        analysis_reports should be a string
        """
        prompt = f"""Analyze the following reports and provide an assessment focused on bearish signals.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Analysis Reports: {analysis_reports}"""
        assessment = self.generate_response(prompt)
        assessment = self.remove_code(assessment)  # Clean the output
        return assessment

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class TraderAgent(Agent):
    def __init__(self, risk_profile="medium"):
        super().__init__("Trader Agent")
        self.risk_profile = risk_profile

    def make_trading_decision(self, debate_log):
        """
        Makes a trading decision based on the debate log.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        debate_log should be a list of dictionaries.
        """
        prompt = f"""Based on the market debate: {json.dumps(debate_log)}, 
        make a trading decision with a {self.risk_profile} risk profile.
        YOU MUST NOT GENERATE ANY PYTHON CODE."""
        decision = self.generate_response(prompt)
        decision = self.remove_code(decision)  # Clean the output
        return decision

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

class RiskManagementTeam(Agent):
    def __init__(self):
        super().__init__("Risk Management Team")

    def assess_risk(self, trading_decision):
        """
        Assesses the risk of a trading decision.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        trading_decision should be a string.
        """
        prompt = f"""Assess the risk associated with the following trading decision: {trading_decision}
        YOU MUST NOT GENERATE ANY PYTHON CODE."""
        risk_assessment = self.generate_response(prompt)
        risk_assessment = self.remove_code(risk_assessment)  # Clean the output
        return risk_assessment

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code
    
class FundManager(Agent):
    def __init__(self):
        super().__init__("Fund Manager")
    
    def approve_trade(self, trading_decision, risk_assessment):
        """
        Approves or rejects a trade based on the decision and risk assessment.
        YOU MUST NOT GENERATE ANY PYTHON CODES.
        trading_decision and risk_assessment should be strings.
        """
        prompt = f"""The trading decision is: {trading_decision} and the risk assessment is: {risk_assessment}. Approve or reject this trade.
        YOU MUST NOT GENERATE ANY PYTHON CODE."""
        approval = self.generate_response(prompt)
        approval = self.remove_code(approval)  # Clean the output
        return approval

    def remove_code(self, text):
        """Removes code blocks from text."""
        code_pattern = re.compile(r"```.*?```", re.DOTALL)  # Matches code blocks
        text_without_code = code_pattern.sub("", text)
        return text_without_code

# --- Example Usage ---
if __name__ == "__main__":
    # Initialize agents
    agents = {
        "market_debate_agent": MarketDebateAgent(),  # Debate orchestrator
        "trader_agent": TraderAgent(),
        "risk_management_team": RiskManagementTeam(),
        "fund_manager": FundManager()
    }

    # Load alpha factors
    alpha_factors = load_alpha_factors(ALPHA_FACTOR_DIR)
    if not alpha_factors:
        logging.warning("No alpha factors loaded. Exiting.")
        exit()

    # Simulate data loading
    company_data = {
        "0001.HK": {"Q1_revenue": 100, "Q2_revenue": 105, "debt": 20}
    }
    social_media_data = "Trending on Twitter: #AAPLBuy strong positive sentiment"
    news_data = "Breaking: AAPL announces new product launch"
    price_data = pd.DataFrame({
        "timestamp": pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05']),
        "price": [150, 152, 155, 153, 156],
        "volume": [1000000, 1050000, 1100000, 950000, 1200000]
    })

    # Agent workflow
    debate_log = agents["market_debate_agent"].conduct_debate(
        company_data, social_media_data, news_data, price_data, alpha_factors)  # Conduct the debate

    # Review alpha factor results
    for entry in debate_log:
        if entry["role"] == "debate_summary":
            alpha_analysis = entry["alpha_analysis"]
            print("Alpha Factor Analysis Results:")
            print(alpha_analysis)

    trading_decision = agents["trader_agent"].make_trading_decision(debate_log)  # Trader uses debate log
    risk_assessment = agents["risk_management_team"].assess_risk(trading_decision)
    approval = agents["fund_manager"].approve_trade(trading_decision, risk_assessment)

    print("Trading Process Completed.")

    # Save logs
    save_logs(agents.values())

2025-03-20 15:11:39,836 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_2.csv'
2025-03-20 15:11:39,838 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_3.csv'
2025-03-20 15:11:39,839 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_1.csv'
2025-03-20 15:11:39,840 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_4.csv'
2025-03-20 15:11:39,842 - INFO - Loaded alpha factors from 'outputs/HK_specific_comprehensive_result_5.csv'
2025-03-20 15:11:44,211 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-20 15:11:44,212 - INFO - Fundamentals Analyst: Generated response - Analyze the following company data and provide a fundamentals analysis report.
        YOU MUST NOT GENERATE ANY PYTHON CODE. Focus on providing a qualitative analysis of the data.
        Company Data: {'AAPL': {'Q1_revenue': 100, 'Q2_revenue': 105, 'debt': 20}}
2025-03-20 15:11:4

Alpha Factor Analysis Results:
Based on the provided data and analysis, I recommend selecting the following alpha factors:

1. **Price Momentum (14 days)**: This factor is relevant for identifying trends in the stock's price over a short period. Since AAPL has been exhibiting upward momentum in recent years, this factor may be useful for confirming that trend.
2. **Mean Reversion (20 days)**: Mean reversion is a fundamental concept in finance, and this factor can help identify if AAPL's price is due for a correction or a return to its historical mean. This factor can provide insights into the stock's potential volatility and risk.
3. **Gross Profit Margin**: As a leading technology company, Apple's gross profit margin is an important metric to track. This factor can help identify any changes in the company's profitability and provide insights into its competitive position.
4. **20-Day Volatility**: This factor can help measure the stock's volatility over a relatively short period. High

2025-03-20 15:12:36,039 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-20 15:12:36,041 - INFO - Trader Agent: Generated response - Based on the market debate: [{"role": "fundamentals_analyst", "report": "**Fundamentals Analysis Report**\n\nBased on the provided company data for Apple Inc. (AAPL), this report provides a qualitative analysis of the company's fundamentals.\n\n**Revenue Growth:**\nThe revenue growth pattern is as follows:\n- Q1 revenue: $100 billion\n- Q2 revenue: $105 billion, indicating an increase of 5% from Q1.\nThis modest growth rate suggests that Apple's sales are still driven by its core product lines, such as iPhones and Macs. However, the slight increase in revenue from one quarter to another implies a stable market position for the company.\n\n**Debt:**\nThe total debt for Apple Inc. is $20 billion. While this amount seems relatively low compared to other large corporations, it can be seen as an indicator of the company's f

Trading Process Completed.


In [36]:
def review_alpha_factor_decision(debate_log):
    """
    Extracts and prints the alpha factor analysis from the debate log.
    """
    for entry in debate_log:
        if "role" in entry and entry["role"] == "debate_summary":
            if "alpha_analysis" in entry:
                return entry["alpha_analysis"]
            else:
                print("Alpha factor analysis key 'alpha_analysis' not found in 'debate_summary' entry.")
                return None  # Or consider raising an exception
    print("No 'debate_summary' entry found in debate log.")
    return None

alpha_factor_decision = review_alpha_factor_decision(debate_log)
if alpha_factor_decision:
    print("Final Alpha Factor Decision:")
    print(alpha_factor_decision)
else:
    print("Alpha factor decision not found in debate log.")

Final Alpha Factor Decision:
Based on the provided data and analysis, I recommend selecting the following alpha factors:

1. **Price Momentum (14 days)**: This factor is relevant for identifying trends in the stock's price over a short period. Since AAPL has been exhibiting upward momentum in recent years, this factor may be useful for confirming that trend.
2. **Mean Reversion (20 days)**: Mean reversion is a fundamental concept in finance, and this factor can help identify if AAPL's price is due for a correction or a return to its historical mean. This factor can provide insights into the stock's potential volatility and risk.
3. **Gross Profit Margin**: As a leading technology company, Apple's gross profit margin is an important metric to track. This factor can help identify any changes in the company's profitability and provide insights into its competitive position.
4. **20-Day Volatility**: This factor can help measure the stock's volatility over a relatively short period. High v

# Data Collection for multi-agents

In [38]:
from openbb import obb

def get_fundamental_data(symbol: str):
    """
    Retrieves fundamental data for a given symbol using OpenBB.
    """
    try:
        financial_statements = obb.equity.financials.income_statement(symbol=symbol)
        # Extract relevant data from financial_statements
        # Example:
        # revenue = financial_statements.revenue
        return financial_statements  # Or return extracted data
    except Exception as e:
        logging.error(f"Error fetching fundamental data for {symbol}: {e}")
        return None

def get_price_data(symbol: str, start_date: str, end_date: str):
    """
    Retrieves historical price data for a symbol using OpenBB.
    """
    try:
        price_data = obb.equity.historical_price.run(
            symbol=symbol,
            start_date=start_date,
            end_date=end_date
        )
        # Convert to DataFrame if needed
        price_df = price_data.df
        return price_df
    except Exception as e:
        logging.error(f"Error fetching price data for {symbol}: {e}")
        return None

def get_news_data(symbol: str, limit: int = 10):
    """
    Retrieves news headlines for a symbol using OpenBB.
    """
    try:
        news = obb.news.headlines(symbol=symbol, limit=limit)
        return news
    except Exception as e:
        logging.error(f"Error fetching news data for {symbol}: {e}")
        return None
    
def get_social_media_data(symbol: str):
    """
    This is a placeholder. You'll need to implement sentiment analysis
    using OpenBB or another library.
    """
    try:
        # Implement sentiment analysis here using OpenBB or other tools
        # Example (Conceptual):
        sentiment = obb.sentiment.get_sentiment(symbol=symbol)
        # return sentiment
        return "Social media sentiment analysis not yet implemented."
    except Exception as e:
        logging.error(f"Error fetching social media data for {symbol}: {e}")
        return "Error fetching social media data."

# In your main section:
social_media_data = get_social_media_data("AAPL")

# In your main section:
company_data = get_fundamental_data("AAPL")  # Example for Apple
price_data = get_price_data("AAPL", "2024-01-01", "2024-01-05")
news_data = get_news_data("AAPL")

print(company_data, price_data, news_data)

2025-03-20 15:45:19,967 - ERROR - Error fetching social media data for AAPL: 'App' object has no attribute 'sentiment'
2025-03-20 15:45:19,968 - ERROR - Error fetching fundamental data for AAPL: 'ROUTER_equity' object has no attribute 'financials'
2025-03-20 15:45:19,969 - ERROR - Error fetching price data for AAPL: 'ROUTER_equity' object has no attribute 'historical_price'
2025-03-20 15:45:19,970 - ERROR - Error fetching news data for AAPL: 'ROUTER_news' object has no attribute 'headlines'


None None None


In [49]:
from openbb import obb

# Login with personal access token
obb.account.login(pat="c7a430dc7df499f151792d5f690297a6")

OpenBBError: Failed to decode Platform token.

In [45]:
from openbb import obb
import pandas as pd
import logging

def get_fundamental_data(symbol: str):
    """
    Retrieves fundamental data for a given symbol using OpenBB.
    """
    try:
        # --- Accessing fundamental data ---
        income_statement_results = obb.equity.fundamental.income(symbol=symbol).results
        balance_sheet_results = obb.equity.fundamental.balance(symbol=symbol).results
        cash_flow_results = obb.equity.fundamental.cash(symbol=symbol).results

        income_statement = income_statement_results.to_df() if hasattr(income_statement_results, 'to_df') else pd.DataFrame()
        balance_sheet = balance_sheet_results.to_df() if hasattr(balance_sheet_results, 'to_df') else pd.DataFrame()
        cash_flow = cash_flow_results.to_df() if hasattr(cash_flow_results, 'to_df') else pd.DataFrame()

        return income_statement, balance_sheet, cash_flow

    except Exception as e:
        logging.error(f"Error fetching fundamental data for {symbol}: {e}")
        return None, None, None

def get_price_data(symbol: str, start_date: str, end_date: str):
    """
    Retrieves historical price data for a symbol using OpenBB.
    """
    try:
        # --- Accessing historical price data ---
        price_data_results = obb.equity.price.historical(symbol=symbol, start_date=start_date, end_date=end_date).results
        price_data = price_data_results.to_df() if hasattr(price_data_results, 'to_df') else pd.DataFrame()
        return price_data
    except Exception as e:
        logging.error(f"Error fetching price data for {symbol}: {e}")
        return None

def get_news_data(symbol: str, limit: int = 10):
    """
    Retrieves news headlines for a symbol using OpenBB.
    """
    try:
        # --- Accessing news headlines ---
        news_results = obb.news.world(symbol=symbol, limit=limit).results
        news = news_results.to_df() if hasattr(news_results, 'to_df') else pd.DataFrame()
        return news
    except Exception as e:
        logging.error(f"Error fetching news data for {symbol}: {e}")
        return None

def get_social_media_data(symbol: str):
    """
    This is a placeholder. You'll need to implement sentiment analysis
    using OpenBB or another library.
    """
    try:
        # --- Sentiment analysis is complex ---
        # --- Placeholder ---
        return "Social media sentiment analysis not yet implemented."
    except Exception as e:
        logging.error(f"Error fetching social media data for {symbol}: {e}")
        return "Error fetching social media data."

# In your main section:
social_media_data = get_social_media_data("AAPL")
print(f"Social Media Data: {social_media_data}\n")

# In your main section:
income_statement, balance_sheet, cash_flow = get_fundamental_data("AAPL")  # Example for Apple
print("Income Statement:\n", income_statement.head() if income_statement is not None else "Could not retrieve Income Statement data.\n")
print("Balance Sheet:\n", balance_sheet.head() if balance_sheet is not None else "Could not retrieve Balance Sheet data.\n")
print("Cash Flow Statement:\n", cash_flow.head() if cash_flow is not None else "Could not retrieve Cash Flow Statement data.\n")

price_data = get_price_data("AAPL", "2024-01-01", "2024-01-05")
print("Price Data:\n", price_data.head() if price_data is not None else "Could not retrieve Price Data.\n")

news_data = get_news_data("AAPL")
print("News Data:\n", news_data.head() if news_data is not None else "Could not retrieve News Data.\n")

Social Media Data: Social media sentiment analysis not yet implemented.

Income Statement:
 Empty DataFrame
Columns: []
Index: []
Balance Sheet:
 Empty DataFrame
Columns: []
Index: []
Cash Flow Statement:
 Empty DataFrame
Columns: []
Index: []


2025-03-20 15:52:06,208 - ERROR - Error fetching news data for AAPL: 
[Error] -> Provider fallback failed.
[Providers]
  * 'benzinga' -> missing credentials
  * 'fmp' -> missing credentials
  * 'intrinio' -> missing credentials
  * 'tiingo' -> missing credentials


Price Data:
 Empty DataFrame
Columns: []
Index: []
News Data:
 Could not retrieve News Data.



In [44]:
# In your main section:
income_statement, balance_sheet, cash_flow = get_fundamental_data("AAPL")

if income_statement is not None:
    print("Income Statement:\n", income_statement.head(), "\n")
else:
    print("Could not retrieve Income Statement data for AAPL.\n")

if balance_sheet is not None:
    print("Balance Sheet:\n", balance_sheet.head(), "\n")
else:
    print("Could not retrieve Balance Sheet data for AAPL.\n")

if cash_flow is not None:
    print("Cash Flow Statement:\n", cash_flow.head(), "\n")
else:
    print("Could not retrieve Cash Flow Statement data for AAPL.\n")

price_data = get_price_data("AAPL", "2024-01-01", "2024-01-05")
if price_data is not None:
    print("Price Data:\n", price_data, "\n")
else:
    print("Could not retrieve Price Data for AAPL.\n")

news_data = get_news_data("AAPL")
if news_data is not None:
    print("News Data:\n", news_data.head())
else:
    print("Could not retrieve News Data for AAPL.\n")

2025-03-20 15:51:31,296 - ERROR - Error fetching fundamental data for AAPL: 'list' object has no attribute 'to_df'


Could not retrieve Income Statement data for AAPL.

Could not retrieve Balance Sheet data for AAPL.

Could not retrieve Cash Flow Statement data for AAPL.



2025-03-20 15:51:32,598 - ERROR - Error fetching price data for AAPL: 'list' object has no attribute 'to_df'
2025-03-20 15:51:32,599 - ERROR - Error fetching news data for AAPL: 'ROUTER_news' object has no attribute 'headlines'


Could not retrieve Price Data for AAPL.

Could not retrieve News Data for AAPL.

